In [0]:
# Loading all the cleaned files (For Demo purpose I have used GitHub to load the datasets)
# Base URL
base_url = "https://raw.githubusercontent.com/sriram1105-m/Customer-Intelligence-Platform/main/data/"

# Dictionary of dataset filenames and variables
datasets = {
    "customers_clean.csv" : "customers_clean_df",
    "products_clean.csv" : "products_clean_df",
    "transactions_clean.csv" : "transactions_clean_df",
    "returns_clean.csv" : "returns_clean_df",
    "support_clean.csv" : "support_clean_df",
    "web_clean.csv" : "web_clean_df"
}

# Load using Pandas and convert to Spark Dataframe
for file_name, var_name in datasets.items():
    pdf = pd.read_csv(base_url + file_name)
    sdf = spark.createDataFrame(pdf)
    globals()[var_name] = sdf

# Quick Row Count check for all datasets
print("\nRow Counts for all Datasets: ")
for var_name in datasets.values():
    df = globals()[var_name]
    print(f"{var_name} : {df.count()} rows")


Row Counts for all Datasets: 
customers_clean_df : 10000 rows
products_clean_df : 500 rows
transactions_clean_df : 50000 rows
returns_clean_df : 1500 rows
support_clean_df : 10000 rows
web_clean_df : 30000 rows


Transactions Dataset

In [0]:
# RFM (Recency, Frequency, Monetary) Metrics for Transactions Dataset

from pyspark.sql import functions as F

# Step 1: Find the latest transaction date in the dataset
max_txn_date = transactions_clean_df.agg(F.max("transaction_date")).collect()[0][0]
print("Latest Transaction Date: ", max_txn_date)

# Step 2: Recency --> Days since last purchase per customer
recency_df = transactions_clean_df.groupBy("customer_id") \
                                  .agg(F.datediff(F.lit(max_txn_date), 
                                                  F.max("transaction_date")).alias("recency_days"))
                                  
# Step 3: Frequency --> Number of transactions per customer
frequency_df = transactions_clean_df.groupBy("customer_id") \
                                    .agg(F.count("transaction_id").alias("frequency"))

# Step 4: Monetary --> Total spend per customer
monetary_df = transactions_clean_df.groupBy("customer_id") \
                                   .agg(F.sum("amount").alias("monetary_value"))

# Step 5: Combine all RFM Metrics into one table
rfm_df = recency_df.join(frequency_df, on = "customer_id", how = "left") \
                   .join(monetary_df, on = "customer_id", how = "left")

display(rfm_df.limit(10))

Latest Transaction Date:  2025-09-11


customer_id,recency_days,frequency,monetary_value
d8a580b8-4c19-46ba-afaa-e07ac2666d3f,69,2,1451.28
59162ae4-44e3-49e2-9c52-3e897c8292b8,97,11,6017.07
c866be64-36d4-45b1-864a-2b1165ae6198,94,6,3343.7299999999996
6acb9ae8-6732-49a0-ad77-f4720e6a9efa,351,5,1663.5
557c1ec2-d512-4500-820f-55e28b1dc300,123,6,2427.1400000000003
96fd27b9-3f67-4748-a90c-281e7eaefc1f,40,1,249.87
5b452ab5-3f2a-4ac5-bd2e-23812c01b811,200,4,2821.76
2130fc55-edc4-4272-b77d-e4c3c35cccb7,248,4,1756.49
899ba871-9497-4f48-beae-6ff5fd1155fa,176,4,3046.5
bc52aab0-71e3-4315-ad57-3c6a4c41c211,58,6,4160.42


In [0]:
# Validating the RFM Metrics
# Check for Null values in RFM metrics
rfm_df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in rfm_df.columns]).show()

# Quick summary statistics for numeric columns
rfm_df.describe(["recency_days", "frequency", "monetary_value"]).show()

+-----------+------------+---------+--------------+
|customer_id|recency_days|frequency|monetary_value|
+-----------+------------+---------+--------------+
|          0|           0|        0|             0|
+-----------+------------+---------+--------------+

+-------+------------------+------------------+------------------+
|summary|      recency_days|         frequency|    monetary_value|
+-------+------------------+------------------+------------------+
|  count|              9941|              9941|              9941|
|   mean|141.62458505180567| 5.029675082989639|2728.3680293733046|
| stddev|132.33072591529515|2.1953454935540835|1590.3342814247806|
|    min|                 0|                 1|               8.2|
|    max|               726|                17|10909.650000000001|
+-------+------------------+------------------+------------------+



In [0]:
# Adding 2 more KPIs to Transaction Dataset
# 1. Average Order Value (AOV)
rfm_df = rfm_df.withColumn("avg_order_value", 
                           F.round(F.col("monetary_value") / F.col("frequency"), 2))

# 2. Preferred Payment Method (Most used Payment method per customer)
payment_method_df = transactions_clean_df.groupBy("customer_id", "payment_method") \
                                         .count() \
                                         .withColumnRenamed("count", "payment_count")

# Get the preferred method --> Max Count per customer
from pyspark.sql.window import Window

window_spec = Window.partitionBy("customer_id").orderBy(F.col("payment_count").desc())
preferred_payment_df = payment_method_df.withColumn("rank", F.rank().over(window_spec)) \
                                        .filter(F.col("rank") == 1) \
                                        .drop("rank", "payment_count")

# Join with RFM Table
rfm_df = rfm_df.join(preferred_payment_df, on = "customer_id", how = "left")

display(rfm_df.limit(10))
                                          

customer_id,recency_days,frequency,monetary_value,avg_order_value,payment_method
d8a580b8-4c19-46ba-afaa-e07ac2666d3f,69,2,1451.28,725.64,Credit Card
59162ae4-44e3-49e2-9c52-3e897c8292b8,97,11,6017.07,547.01,Credit Card
c866be64-36d4-45b1-864a-2b1165ae6198,94,6,3343.7299999999996,557.29,Wallet
6acb9ae8-6732-49a0-ad77-f4720e6a9efa,351,5,1663.5,332.7,Wallet
557c1ec2-d512-4500-820f-55e28b1dc300,123,6,2427.1400000000003,404.52,Credit Card
96fd27b9-3f67-4748-a90c-281e7eaefc1f,40,1,249.87,249.87,UPI
5b452ab5-3f2a-4ac5-bd2e-23812c01b811,200,4,2821.76,705.44,Credit Card
2130fc55-edc4-4272-b77d-e4c3c35cccb7,248,4,1756.49,439.12,Wallet
899ba871-9497-4f48-beae-6ff5fd1155fa,176,4,3046.5,761.63,UPI
bc52aab0-71e3-4315-ad57-3c6a4c41c211,58,6,4160.42,693.4,Wallet


Returns KPIs

In [0]:
from pyspark.sql import functions as F

# Step 1: Total returns per customer
returns_count_df = returns_clean_df.groupBy("customer_id") \
                                   .agg(F.count("return_id").alias("total_returns"))

# Step 2: Join returns with transactions count for return rate
returns_rate_df = returns_count_df.join(rfm_df.select("customer_id", "frequency"),
                                        on = "customer_id",
                                        how = "left"
                                        ).withColumn(
                                            "return_rate",
                                            F.round(F.col("total_returns") / F.col("frequency"), 2)
                                        )

# Step 3: Join back to RFM Table
rfm_df = rfm_df.join(returns_rate_df.select("customer_id", "total_returns", "return_rate"),
                                            on = "customer_id", how = "left")

# Fill null return_rate with 0 for customers with no returns
rfm_df = rfm_df.fillna({"total_returns" : 0, "return_rate" : 0})

display(rfm_df.limit(10))

customer_id,recency_days,frequency,monetary_value,avg_order_value,payment_method,total_returns,return_rate
d8a580b8-4c19-46ba-afaa-e07ac2666d3f,69,2,1451.28,725.64,Credit Card,0,0.0
59162ae4-44e3-49e2-9c52-3e897c8292b8,97,11,6017.07,547.01,Credit Card,0,0.0
c866be64-36d4-45b1-864a-2b1165ae6198,94,6,3343.7299999999996,557.29,Wallet,0,0.0
6acb9ae8-6732-49a0-ad77-f4720e6a9efa,351,5,1663.5,332.7,Wallet,0,0.0
557c1ec2-d512-4500-820f-55e28b1dc300,123,6,2427.1400000000003,404.52,Credit Card,0,0.0
96fd27b9-3f67-4748-a90c-281e7eaefc1f,40,1,249.87,249.87,UPI,0,0.0
5b452ab5-3f2a-4ac5-bd2e-23812c01b811,200,4,2821.76,705.44,Credit Card,1,0.25
2130fc55-edc4-4272-b77d-e4c3c35cccb7,248,4,1756.49,439.12,Wallet,0,0.0
899ba871-9497-4f48-beae-6ff5fd1155fa,176,4,3046.5,761.63,UPI,0,0.0
bc52aab0-71e3-4315-ad57-3c6a4c41c211,58,6,4160.42,693.4,Wallet,0,0.0


Web Activity KPIs


In [0]:
from pyspark.sql.functions import col, countDistinct, lit, sum as Fsum, when

# Step 1: Calculate Total Web Sessions per customer
web_sessions_df = web_clean_df.groupBy("customer_id") \
                              .agg(countDistinct("session_id").alias("total_sessions"))

# Step 2: Active days per customer
web_days_df = web_clean_df.withColumn("activity_date", F.to_date("timestamp")) \
                          .groupBy("customer_id") \
                          .agg(countDistinct("activity_date").alias("active_days"))

# Step 3: Engagement Score (assign weights per activity_type)
engagement_df = web_clean_df.withColumn(
    "activity_score",
    when(col("activity_type") == "view", lit(1))
    .when(col("activity_type") == "click", lit(2))
    .when(col("activity_type") == "cart", lit(3))
    .when(col("activity_type") == "purchase", lit(4))
    .otherwise(lit(0))
).groupBy("customer_id").agg(Fsum("activity_score").alias("engagement_score"))

# Step 4: Join all Web KPIs
web_kpis_df = web_sessions_df.join(web_days_df, on = "customer_id", how = "left") \
                             .join(engagement_df, on = "customer_id", how = "left")

# Step 5: Join with RFM Table
rfm_df = rfm_df.join(web_kpis_df, on = "customer_id", how = "left").fillna(0)

display(rfm_df.limit(10))


customer_id,recency_days,frequency,monetary_value,avg_order_value,payment_method,total_returns,return_rate,total_sessions,active_days,engagement_score
d8a580b8-4c19-46ba-afaa-e07ac2666d3f,69,2,1451.28,725.64,Credit Card,0,0.0,5,5,0
59162ae4-44e3-49e2-9c52-3e897c8292b8,97,11,6017.07,547.01,Credit Card,0,0.0,1,1,0
c866be64-36d4-45b1-864a-2b1165ae6198,94,6,3343.7299999999996,557.29,Wallet,0,0.0,2,2,0
6acb9ae8-6732-49a0-ad77-f4720e6a9efa,351,5,1663.5,332.7,Wallet,0,0.0,3,3,0
557c1ec2-d512-4500-820f-55e28b1dc300,123,6,2427.1400000000003,404.52,Credit Card,0,0.0,2,2,0
96fd27b9-3f67-4748-a90c-281e7eaefc1f,40,1,249.87,249.87,UPI,0,0.0,3,3,0
5b452ab5-3f2a-4ac5-bd2e-23812c01b811,200,4,2821.76,705.44,Credit Card,1,0.25,2,2,0
2130fc55-edc4-4272-b77d-e4c3c35cccb7,248,4,1756.49,439.12,Wallet,0,0.0,0,0,0
899ba871-9497-4f48-beae-6ff5fd1155fa,176,4,3046.5,761.63,UPI,0,0.0,1,1,0
bc52aab0-71e3-4315-ad57-3c6a4c41c211,58,6,4160.42,693.4,Wallet,0,0.0,2,2,0


Support Ticket KPIs

In [0]:
from pyspark.sql.functions import datediff

# Step 1: Total tickets per customer
tickets_count_df = support_clean_df.groupBy("customer_id") \
                                   .agg(F.count("ticket_id").alias("total_tickets"))

# Step 2: Average resolution time (in days)
resolution_time_df = support_clean_df.withColumn("resolution_days",
                                                 datediff("resolved_date", "opened_date")).groupBy("customer_id") \
                                                                                          .agg(F.avg("resolution_days").alias("avg_resolution_days"))

# Step 3: Open Tickets (Status != Resolved)
open_tickets_df = support_clean_df.filter(F.col("status") != "Resolved") \
                                  .groupBy("customer_id") \
                                  .agg(F.count("ticket_id").alias("open_tickets"))

# Step 4: Join all Support KPIs
support_kpis_df = tickets_count_df.join(resolution_time_df, on = "customer_id", how = "left") \
                                  .join(open_tickets_df, on = "customer_id", how = "left")

# Step 5: Join with RFM Table
rfm_df = rfm_df.join(support_kpis_df, on = "customer_id", how = "left").fillna(0)

display(rfm_df.limit(10))

customer_id,recency_days,frequency,monetary_value,avg_order_value,payment_method,total_returns,return_rate,total_sessions,active_days,engagement_score,total_tickets,avg_resolution_days,open_tickets
d8a580b8-4c19-46ba-afaa-e07ac2666d3f,69,2,1451.28,725.64,Credit Card,0,0.0,5,5,0,2,9.0,1
59162ae4-44e3-49e2-9c52-3e897c8292b8,97,11,6017.07,547.01,Credit Card,0,0.0,1,1,0,0,0.0,0
c866be64-36d4-45b1-864a-2b1165ae6198,94,6,3343.7299999999996,557.29,Wallet,0,0.0,2,2,0,1,0.0,1
6acb9ae8-6732-49a0-ad77-f4720e6a9efa,351,5,1663.5,332.7,Wallet,0,0.0,3,3,0,1,23.0,1
557c1ec2-d512-4500-820f-55e28b1dc300,123,6,2427.1400000000003,404.52,Credit Card,0,0.0,2,2,0,1,0.0,1
96fd27b9-3f67-4748-a90c-281e7eaefc1f,40,1,249.87,249.87,UPI,0,0.0,3,3,0,0,0.0,0
5b452ab5-3f2a-4ac5-bd2e-23812c01b811,200,4,2821.76,705.44,Credit Card,1,0.25,2,2,0,2,0.0,2
2130fc55-edc4-4272-b77d-e4c3c35cccb7,248,4,1756.49,439.12,Wallet,0,0.0,0,0,0,1,8.0,0
899ba871-9497-4f48-beae-6ff5fd1155fa,176,4,3046.5,761.63,UPI,0,0.0,1,1,0,1,3.0,1
bc52aab0-71e3-4315-ad57-3c6a4c41c211,58,6,4160.42,693.4,Wallet,0,0.0,2,2,0,0,0.0,0


Product KPIs

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Step 1: Join Transactions with Products to get category and brand info
txn_products_df = transactions_clean_df.join(products_clean_df, on = "product_id", how = "left")

# Step 2: Total Spend per category per customer
category_spend_df = txn_products_df.groupBy("customer_id", "category") \
                                   .agg(F.sum("amount").alias("category_spend"))

# Step 3: Get Top Category per customer (highest spend)
category_window = Window.partitionBy("customer_id").orderBy(F.col("category_spend").desc())

top_category_df = category_spend_df.withColumn("rank", F.rank().over(category_window)) \
                                   .filter(F.col("rank") == 1) \
                                   .drop("rank", "category_spend") \
                                   .withColumnRenamed("category", "top_category")

# Step 4: Total Spend per brand per customer
brand_spend_df = txn_products_df.groupBy("customer_id", "brand") \
                                .agg(F.sum("amount").alias("brand_spend"))

# Step 5: Get Top brand per customer
brand_window = Window.partitionBy("customer_id").orderBy(F.col("brand_spend").desc())
                            
top_brand_df = brand_spend_df.withColumn("rank", F.rank().over(brand_window)) \
                             .filter(F.col("rank") == 1) \
                             .drop("rank", "brand_spend") \
                             .withColumnRenamed("brand", "top_brand")

# Step 6: Join Top Category & Top Brand with RFM Table
rfm_df = rfm_df.join(top_category_df, on = "customer_id", how = "left") \
               .join(top_brand_df, on = "customer_id", how = "left")

display(rfm_df.limit(10))

customer_id,recency_days,frequency,monetary_value,avg_order_value,payment_method,total_returns,return_rate,total_sessions,active_days,engagement_score,total_tickets,avg_resolution_days,open_tickets,top_category,top_brand
d8a580b8-4c19-46ba-afaa-e07ac2666d3f,69,2,1451.28,725.64,Credit Card,0,0.0,5,5,0,2,9.0,1,Grocery,Smith LLC
59162ae4-44e3-49e2-9c52-3e897c8292b8,97,11,6017.07,547.01,Credit Card,0,0.0,1,1,0,0,0.0,0,Sports,"Le, Gonzales and Robinson"
c866be64-36d4-45b1-864a-2b1165ae6198,94,6,3343.7299999999996,557.29,Wallet,0,0.0,2,2,0,1,0.0,1,Home,"Le, Gonzales and Robinson"
6acb9ae8-6732-49a0-ad77-f4720e6a9efa,351,5,1663.5,332.7,Wallet,0,0.0,3,3,0,1,23.0,1,Clothing,Chavez-Turner
557c1ec2-d512-4500-820f-55e28b1dc300,123,6,2427.1400000000003,404.52,Credit Card,0,0.0,2,2,0,1,0.0,1,Clothing,"Green, Hall and Scott"
96fd27b9-3f67-4748-a90c-281e7eaefc1f,40,1,249.87,249.87,UPI,0,0.0,3,3,0,0,0.0,0,Home,Meyers-Taylor
5b452ab5-3f2a-4ac5-bd2e-23812c01b811,200,4,2821.76,705.44,Credit Card,1,0.25,2,2,0,2,0.0,2,Grocery,"Kim, Barrera and Griffin"
2130fc55-edc4-4272-b77d-e4c3c35cccb7,248,4,1756.49,439.12,Wallet,0,0.0,0,0,0,1,8.0,0,Grocery,Spears PLC
899ba871-9497-4f48-beae-6ff5fd1155fa,176,4,3046.5,761.63,UPI,0,0.0,1,1,0,1,3.0,1,Clothing,"Compton, Cordova and Martinez"
bc52aab0-71e3-4315-ad57-3c6a4c41c211,58,6,4160.42,693.4,Wallet,0,0.0,2,2,0,0,0.0,0,Electronics,Chambers Ltd


Customer 360 Table

In [0]:
# Final Customer 360 Table

customer_360_df = rfm_df.select(
    "customer_id",
    "recency_days",
    "frequency",
    "monetary_value",
    "avg_order_value",
    "payment_method",
    "total_returns",
    "return_rate",
    "total_sessions",
    "active_days",
    "engagement_score",
    "total_tickets",
    "avg_resolution_days",
    "open_tickets",
    "top_category",
    "top_brand"
)

display(customer_360_df.limit(10))

customer_id,recency_days,frequency,monetary_value,avg_order_value,payment_method,total_returns,return_rate,total_sessions,active_days,engagement_score,total_tickets,avg_resolution_days,open_tickets,top_category,top_brand
d8a580b8-4c19-46ba-afaa-e07ac2666d3f,69,2,1451.28,725.64,Credit Card,0,0.0,5,5,0,2,9.0,1,Grocery,Smith LLC
59162ae4-44e3-49e2-9c52-3e897c8292b8,97,11,6017.07,547.01,Credit Card,0,0.0,1,1,0,0,0.0,0,Sports,"Le, Gonzales and Robinson"
c866be64-36d4-45b1-864a-2b1165ae6198,94,6,3343.7299999999996,557.29,Wallet,0,0.0,2,2,0,1,0.0,1,Home,"Le, Gonzales and Robinson"
6acb9ae8-6732-49a0-ad77-f4720e6a9efa,351,5,1663.5,332.7,Wallet,0,0.0,3,3,0,1,23.0,1,Clothing,Chavez-Turner
557c1ec2-d512-4500-820f-55e28b1dc300,123,6,2427.1400000000003,404.52,Credit Card,0,0.0,2,2,0,1,0.0,1,Clothing,"Green, Hall and Scott"
96fd27b9-3f67-4748-a90c-281e7eaefc1f,40,1,249.87,249.87,UPI,0,0.0,3,3,0,0,0.0,0,Home,Meyers-Taylor
5b452ab5-3f2a-4ac5-bd2e-23812c01b811,200,4,2821.76,705.44,Credit Card,1,0.25,2,2,0,2,0.0,2,Grocery,"Kim, Barrera and Griffin"
2130fc55-edc4-4272-b77d-e4c3c35cccb7,248,4,1756.49,439.12,Wallet,0,0.0,0,0,0,1,8.0,0,Grocery,Spears PLC
899ba871-9497-4f48-beae-6ff5fd1155fa,176,4,3046.5,761.63,UPI,0,0.0,1,1,0,1,3.0,1,Clothing,"Compton, Cordova and Martinez"
bc52aab0-71e3-4315-ad57-3c6a4c41c211,58,6,4160.42,693.4,Wallet,0,0.0,2,2,0,0,0.0,0,Electronics,Chambers Ltd


In [0]:
# Saving the Customer 360 Table

customer_360_df.toPandas().to_csv("customer_360.csv", index=False)

In [0]:
# Quick Insights
display(customer_360_df.orderBy(F.col("monetary_value").desc()).limit(10))

customer_id,recency_days,frequency,monetary_value,avg_order_value,payment_method,total_returns,return_rate,total_sessions,active_days,engagement_score,total_tickets,avg_resolution_days,open_tickets,top_category,top_brand
d551c651-ed73-437b-a0fa-0431eec32330,43,17,10909.650000000001,641.74,Credit Card,1,0.06,3,3,0,0,0.0,0,Clothing,Hendrix-Ruiz
4b0ca07d-5d92-4ebc-9cdd-a8d5e7f4de7d,22,12,10537.27,878.11,UPI,1,0.08,2,2,0,2,16.0,1,Clothing,Spears PLC
ef4e70f5-7dce-4781-a520-d07f58ed36d5,12,9,10223.07,1135.9,Credit Card,0,0.0,2,2,0,1,0.0,1,Grocery,Shaw Inc
3dc1ac29-f1a2-4eba-94d3-b75bf01c47e5,30,12,10063.800000000001,838.65,UPI,0,0.0,6,6,0,1,30.0,0,Sports,"Ingram, Harris and Sanchez"
68ab34bd-80d7-46e3-a92e-5abca973d0c8,14,13,9888.61,760.66,Cash,0,0.0,9,9,0,1,1.0,1,Sports,"Hines, Norris and Thompson"
21375e94-505f-4957-b1ce-fe0cd9879ba0,15,12,9831.39,819.28,Wallet,0,0.0,4,4,0,0,0.0,0,Electronics,"Chang, Quinn and Martinez"
ee660523-efb6-4261-b1b9-aa1fabc06d2a,44,11,9701.92,881.99,Cash,0,0.0,4,4,0,0,0.0,0,Home,Salazar PLC
567c1b55-b9d0-4030-963e-72688fd8b569,58,10,9593.329999999998,959.33,Credit Card,1,0.1,7,7,0,1,24.0,1,Sports,"Ball, Freeman and Bryant"
5f59d385-aa45-41b9-8f95-285a5f4f1e1a,102,11,9274.27,843.12,Cash,2,0.18,2,2,0,1,25.0,0,Grocery,"Ingram, Bailey and Acosta"
5f59d385-aa45-41b9-8f95-285a5f4f1e1a,102,11,9274.27,843.12,UPI,2,0.18,2,2,0,1,25.0,0,Grocery,"Ingram, Bailey and Acosta"
